# TP0

## **1.**
A interação entre as entidades 'Emitter' e 'Receiver' é feita com 2 threads que comunicam a partir de uma Queue bloqueante.

Alguns imports relevantes para o processo:

In [ ]:
from threading import Thread
from queue import Queue
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import os
from cryptography.hazmat.primitives.ciphers import (Cipher, algorithms, modes)

Primeiro defenimos as funções para cifrar e decifrar as mensagens.

In [ ]:
def encrypt(message,key):
    iv = os.urandom(12)
    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv)
    ).encryptor()
    ciphertext = encryptor.update(message) + encryptor.finalize()
    return (iv,ciphertext,encryptor.tag)

def decrypt(iv,cipher,key,tag):
    decryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv, tag),
    ).decryptor()
    return decryptor.update(cipher) + decryptor.finalize()

Para o 'Emitter' temos uma thread que envia ao 'Receiver' o resultado da cifragem da mensagem "Hello World".

In [ ]:
# Emitter thread function
def emitter(queue):
    global key
    msg = b"Hello World!"
    iv_ct_tag = encrypt(msg,key)
    queue.put(iv_ct_tag)
    print("[Emitter] Sent > {}".format(iv_ct_tag[1]))

Para o 'Receiver' temos uma thread que irá receber e decifrar o criptogama que o 'Emitter' enviou.

In [ ]:
# Receiver thread function
def receiver(queue):
    global key
    iv,ct,tag = queue.get()
    print("[Receiver] Received > {}".format(ct))
    msg = decrypt(iv,ct,key,tag)
    print("[Receiver] Decrypted > {}".format(msg))

Para gerar a chave criptográfica para utilizar na cifragem simétrica lemos do STDIN os bytes de uma string que vai ser usada como password.

In [ ]:
pwd = bytes(input("Shared Password > "),"utf-8")

E derivamos a chave a ser utilizada usando PBKDF, e obtemos a chave na variável 'key'.

In [ ]:
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=16,
    salt=b"\x00"*16,
    iterations=100000
)
key = kdf.derive(pwd)

Para começar toda a interação entre o 'Emitter' e 'Receiver' criamos as threads com as funções e argumentos respetivos e começamos as mesmas.

In [ ]:
q = Queue(5)
    
# Create emitter and receiver threads
e = Thread(target=emitter,args=(q,))
r = Thread(target=receiver,args=(q,))
    
# Start both threads
e.start()
r.start()

# Wait for them to finish to exit program
e.join()
r.join()
print("[INFO] > Finished program execution")